In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ansys.mapdl.core import launch_mapdl

In [2]:
mapdl = launch_mapdl()

In [3]:
# Example analysis of 200 mm x 100 mm plate with center hole diameter of 45 mm and satellite holes of 30 mm at x = 50 , 150 mm

In [4]:
mapdl.clear("NOSTART")
mapdl.prep7()
mapdl.units("SI")  # SI-International system (m, kg, s, K).

# Define a PLANE183 element type with thickness
mapdl.et(1, "PLANE183", kop3=3)
mapdl.r(1, 0.001)  # Thickness of 1 mm

# Define a material (nominal steel in SI)
mapdl.mp("EX", 1, 210e9)  # Elastic moduli in Pa (kg/(m*s**2))
mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
mapdl.mp("NUXY", 1, 0.3)  # Poisson's Ratio
mapdl.emodif("ALL", "MAT", 1)

*** NOTE ***                            CP =       0.375   TIME= 23:39:22
 No elements defined.

In [5]:
# Geometry
    
# Create a rectangular area with the hole in the middle
center_hole_dia = 0.045 # Center hole diameter = 45 mm
sat_holes_dia = 0.03 # Satellite holes diameter = 30 mm
sat_holes_loc = 0.05 # Left satellite holes location at x = 50 mm along y = 50 mm axis

# Create the rectangle
rect_anum = mapdl.blc4(width = 0.2, height = 0.1) # 200 mm x 100 mm

# Create a circle in the middle of the rectangle
circ_anum = mapdl.cyl4(0.1, 0.05, center_hole_dia/2)
    
# Create a satellite circle at locations x = 50 mm along y = 50 mm axis
sat_circ_anum = mapdl.cyl4(sat_holes_loc ,0.05, sat_holes_dia/2)
sat_circ_anum2 = mapdl.cyl4(0.2-sat_holes_loc ,0.05, sat_holes_dia/2) # (200 mm - left hole location) = right hole location

# Cut the circle out of the rectangle.
area1= mapdl.asba(rect_anum, circ_anum)
area2 = mapdl.asba(area1, sat_circ_anum)
plate_with_hole_anum = mapdl.asba(area2, sat_circ_anum2)

In [6]:
mapdl.lplot(cpos="xy", line_width=3, font_size=26, color_lines=True, background="w")

Widget(value="<iframe src='http://localhost:55576/index.html?ui=P_0x1efd00ee940_0&reconnect=auto' style='width…

In [7]:
# Meshing

# Mesh the plate using a higher density near the hole and a lower density for the remainder of the plate
    
mapdl.aclear("all")
  
# Ensure there are at least 100 elements around the hole
hole_esize = np.pi * center_hole_dia / 100 
sat_hole_esize = np.pi * sat_holes_dia / 100 
plate_esize = 0.01
    
# Increased the density of the mesh at the center
mapdl.lsel("S", "LINE", vmin=5, vmax=8) # center hole
mapdl.lesize("ALL", hole_esize, kforc=1)
mapdl.lsel("S", "LINE", vmin=9, vmax=40) # satellite holes
mapdl.lesize("ALL", sat_hole_esize, kforc=1)
mapdl.lsel("ALL")

mapdl.esize(plate_esize)
mapdl.amesh(plate_with_hole_anum)
mapdl.mesh

ANSYS Mesh
  Number of Nodes:              5850
  Number of Elements:           1834
  Number of Element Types:      1
  Number of Node Components:    0
  Number of Element Components: 0

In [8]:
mapdl.eplot(vtk=True,cpos="xy",show_edges=True,show_axes=False,line_width=2,background="w",)

Widget(value="<iframe src='http://localhost:55576/index.html?ui=P_0x1efd1889a60_1&reconnect=auto' style='width…

In [9]:
# Fix the left-hand side of the plate in the X direction
mapdl.nsel("S", "LOC", "X", 0)
mapdl.d("ALL", "UX")

# Fix a single node on the left-hand side of the plate in the Y direction
mapdl.nsel("R", "LOC", "Y", 0.1 / 2)
assert mapdl.mesh.n_node == 1
mapdl.d("ALL", "UY")

# Apply a force on the right-hand side of the plate.

# Select the right-hand side of the plate.
mapdl.nsel("S", "LOC", "X", 0.2)

# Couple the DOF for these nodes
mapdl.cp(5, "UX", "ALL")

# Select a single node in this set and apply a force to it
mapdl.nsel("r", "loc", "y", 0.1 / 2)
mapdl.f("ALL", "FX", 1000) # Apply force of 1000 N

# Be sure to select all nodes again to solve the entire solution
mapdl.allsel()

SELECT ALL ENTITIES OF TYPE= ALL  AND BELOW

In [10]:
# Solve the Static Problem

mapdl.run("/SOLU")
mapdl.antype("STATIC")
output=mapdl.solve()
print(output)

*****  MAPDL SOLVE    COMMAND  *****

 *** NOTE ***                            CP =       3.469   TIME= 23:39:37
 There is no title defined for this analysis.                            

 *** SELECTION OF ELEMENT TECHNOLOGIES FOR APPLICABLE ELEMENTS ***
                ---GIVE SUGGESTIONS ONLY---

 ELEMENT TYPE         1 IS PLANE183 WITH PLANE STRESS OPTION. NO SUGGESTION IS
 AVAILABLE.



 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2023 R1          23.1     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   23:39:37  MAR 26, 2023 CP=      3.469

                                                                               



                       S O L U T I O N   O P T I O N S

   PROBLEM DIMENSIONALITY. . . . . . . . . . . . .2-D                  
   DEGREES OF FREEDOM. . . . . . UX   UY  
   ANALYSIS TYPE . . . . . . . . . . . . . . . . .STATIC (STEADY-STATE)
   GLOBALLY ASSEMBLED MATRIX . . . . . . . . . . .SYMMETRIC  

 *** NOTE ***

In [11]:
# Post-Processing

# Grab the stress from the result
result = mapdl.result
nnum, stress = result.principal_nodal_stress(0)
von_mises = stress[:, -1]
max_stress = np.nanmax(von_mises)

result.plot_principal_nodal_stress(0,"SEQV",lighting=False,cpos="xy",background="w",text_color="k",add_text=False,)

Widget(value="<iframe src='http://localhost:55576/index.html?ui=P_0x1efd41d4370_2&reconnect=auto' style='width…

In [12]:
# Compare to the "far field" stress by getting the mean value of the stress at the wall
mask = result.mesh.nodes[:, 0] == 0.2
far_field_stress = np.nanmean(von_mises[mask])

# Adjust by the cross sectional area at the hole
adj = 0.1 / (0.1 - 0.045) # 100 mm plate length, 45 mm center hole diameter
stress_adj = far_field_stress * adj
stress_concentration = max_stress / stress_adj

In [13]:
print(f'Far field stress = {far_field_stress/1000000:.2f} MPa')
print(f'Maximum stress = {max_stress/1000000:.2f} MPa')
print(f'Stress concentration factor= {stress_concentration:.4f}')

Far field stress = 9.92 MPa
Maximum stress = 36.24 MPa
Stress concentration factor= 2.0097


In [14]:
# Create a function to compute the stress concentration for any 2D plate with a fixed center hole and loaded with an axial force
# With two symmetric satellite holes with various sizes and locations along y = 50 mm axis
# Including the increasing of mesh density around the holes to reach convergence of the stress concentration value < 1 percents

In [15]:
def compute_stress_con(sat_holes_dia,sat_holes_loc):
    
    converge = 1
    iteration = 1
    stress_concentration = 1
    
    while converge >= 0.01: # less than 1 percent of the stress concentration convergence is needed
        mapdl.clear("NOSTART")
        mapdl.prep7()
        mapdl.units("SI")  # SI-International system (m, kg, s, K).

        # Define a PLANE183 element type with thickness
        mapdl.et(1, "PLANE183", kop3=3)
        mapdl.r(1, 0.001)  # Thickness of 1 mm

        # Define a material (nominal steel in SI)
        mapdl.mp("EX", 1, 210e9)  # Elastic moduli in Pa (kg/(m*s**2))
        mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
        mapdl.mp("NUXY", 1, 0.3)  # Poisson's Ratio
        mapdl.emodif("ALL", "MAT", 1)

        # Geometry
    
        # Create a rectangular area with the hole in the middle
        center_hole_dia = 0.045 # center hole diameter = 45 mm
    
        # Create the rectangle
        rect_anum = mapdl.blc4(width = 0.2, height = 0.1) # 200 mm x 100 mm

        # Create a circle in the middle of the rectangle
        circ_anum = mapdl.cyl4(0.1, 0.05, center_hole_dia/2)
    
        # Create a satellite circle at certain locations along y = 50 mm axis
        sat_circ_anum = mapdl.cyl4(sat_holes_loc ,0.05, sat_holes_dia/2)
        sat_circ_anum2 = mapdl.cyl4(0.2-sat_holes_loc ,0.05, sat_holes_dia/2) # 200 mm - left hole location = right hole location

        # Cut the circle out of the rectangle.
        area1= mapdl.asba(rect_anum, circ_anum)
        area2 = mapdl.asba(area1, sat_circ_anum)
        plate_with_hole_anum = mapdl.asba(area2, sat_circ_anum2)   
    
        # Meshing

        # Mesh the plate using a higher density near the hole and a lower density for the remainder of the plate
    
        mapdl.aclear("all")
  
        # Ensure there are at least 100 elements around the hole at beginning then increase 50 elements per iteration
        hole_esize = np.pi * center_hole_dia / ((iteration*50)+50) 
        sat_hole_esize = np.pi * sat_holes_dia / ((iteration*50)+50)  
        plate_esize = 0.01
    
        # Increased the density of the mesh at the center
        mapdl.lsel("S", "LINE", vmin=5, vmax=8) # center hole
        mapdl.lesize("ALL", hole_esize, kforc=1)
        mapdl.lsel("S", "LINE", vmin=9, vmax=40) # satellite holes
        mapdl.lesize("ALL", sat_hole_esize, kforc=1)
        mapdl.lsel("ALL")

        mapdl.esize(plate_esize)
        mapdl.amesh(plate_with_hole_anum)

        # Boundary Conditions

        # Fix the left-hand side of the plate in the X direction
        mapdl.nsel("S", "LOC", "X", 0)
        mapdl.d("ALL", "UX")

        # Fix a single node on the left-hand side of the plate in the Y direction
        mapdl.nsel("R", "LOC", "Y", 0.1 / 2)
        assert mapdl.mesh.n_node == 1
        mapdl.d("ALL", "UY")

        # Apply a force on the right-hand side of the plate.
        
        # Select the right-hand side of the plate.
        mapdl.nsel("S", "LOC", "X", 0.2)

        # Couple the DOF for these nodes
        mapdl.cp(5, "UX", "ALL")

        # Select a single node in this set and apply a force to it
        mapdl.nsel("r", "loc", "y", 0.1 / 2)
        mapdl.f("ALL", "FX", 1000)

        # Be sure to select all nodes again to solve the entire solution
        mapdl.allsel()

        # Solve the Static Problem

        mapdl.run("/SOLU")
        mapdl.antype("STATIC")
        mapdl.solve()

        # Post-Processing

        # Grab the stress from the result
        result = mapdl.result
        nnum, stress = result.principal_nodal_stress(0)
        von_mises = stress[:, -1]
        max_stress = np.nanmax(von_mises)

        # Compare to the "far field" stress by getting the mean value of the stress at the wall
        mask = result.mesh.nodes[:, 0] == 0.2
        far_field_stress = np.nanmean(von_mises[mask])

        # Adjust by the cross sectional area at the hole
        adj = 0.1 / (0.1 - 0.045) # 100 mm plate length, 45 mm center hole diameter
        stress_adj = far_field_stress * adj
        
        converge = abs(stress_concentration - (max_stress / stress_adj))/stress_concentration
        
        stress_concentration = max_stress / stress_adj
        
        iteration +=1
        
    return stress_concentration,iteration-1, converge

In [16]:
Sat_hole_dia=[i for i in range (4,41,2)] # diameter of satellite holes start from 4 to 40 mm with 2 mm increasing step

List=[]

for hole_dia in Sat_hole_dia:
    location = 100
    # Margin = distance from left wall to left radius of the left satellite hole
    # Margin should finally located between 0.005 to 0.076 (x = 5 mm to 76 mm) 
    # The left outer radius of center hole is locate at 77.5 mm
    # Satellite holes should located atleast out of center holes radius but not too far to the edge of plate)
    
    while location-(hole_dia/2)>76:
        location -= 1

    while location-(hole_dia/2)>=5:
        if 0 < 77.5-(location+(hole_dia/2)) < 2: # Check if the distance between satellite and center hole is less than 2 mm ?
            # If the right outer radius of the left satellite hole is too close to the center hole, skip this location
            location -= 1
        else:
            List.append([hole_dia/1000,location/1000]) # Satellite holes sizes and locations for iterations
            location -= 1

In [17]:
len(List)

1330

In [18]:
Kt_tension=[]
Number_of_iter=[]
i=1
for size,loc in List:
    a,b,c=compute_stress_con(size,loc)
    Kt_tension.append(a)
    Number_of_iter.append([b,c])
    print(f'finished {i}/{len(List)} simulation ...')
    i+=1  

finished 1/1330 simulation ...
finished 2/1330 simulation ...
finished 3/1330 simulation ...
finished 4/1330 simulation ...
finished 5/1330 simulation ...
finished 6/1330 simulation ...
finished 7/1330 simulation ...
finished 8/1330 simulation ...
finished 9/1330 simulation ...
finished 10/1330 simulation ...
finished 11/1330 simulation ...
finished 12/1330 simulation ...
finished 13/1330 simulation ...
finished 14/1330 simulation ...
finished 15/1330 simulation ...
finished 16/1330 simulation ...
finished 17/1330 simulation ...
finished 18/1330 simulation ...
finished 19/1330 simulation ...
finished 20/1330 simulation ...
finished 21/1330 simulation ...
finished 22/1330 simulation ...
finished 23/1330 simulation ...
finished 24/1330 simulation ...
finished 25/1330 simulation ...
finished 26/1330 simulation ...
finished 27/1330 simulation ...
finished 28/1330 simulation ...
finished 29/1330 simulation ...
finished 30/1330 simulation ...
finished 31/1330 simulation ...
finished 32/1330 

finished 253/1330 simulation ...
finished 254/1330 simulation ...
finished 255/1330 simulation ...
finished 256/1330 simulation ...
finished 257/1330 simulation ...
finished 258/1330 simulation ...
finished 259/1330 simulation ...
finished 260/1330 simulation ...
finished 261/1330 simulation ...
finished 262/1330 simulation ...
finished 263/1330 simulation ...
finished 264/1330 simulation ...
finished 265/1330 simulation ...
finished 266/1330 simulation ...
finished 267/1330 simulation ...
finished 268/1330 simulation ...
finished 269/1330 simulation ...
finished 270/1330 simulation ...
finished 271/1330 simulation ...
finished 272/1330 simulation ...
finished 273/1330 simulation ...
finished 274/1330 simulation ...
finished 275/1330 simulation ...
finished 276/1330 simulation ...
finished 277/1330 simulation ...
finished 278/1330 simulation ...
finished 279/1330 simulation ...
finished 280/1330 simulation ...
finished 281/1330 simulation ...
finished 282/1330 simulation ...
finished 2

finished 502/1330 simulation ...
finished 503/1330 simulation ...
finished 504/1330 simulation ...
finished 505/1330 simulation ...
finished 506/1330 simulation ...
finished 507/1330 simulation ...
finished 508/1330 simulation ...
finished 509/1330 simulation ...
finished 510/1330 simulation ...
finished 511/1330 simulation ...
finished 512/1330 simulation ...
finished 513/1330 simulation ...
finished 514/1330 simulation ...
finished 515/1330 simulation ...
finished 516/1330 simulation ...
finished 517/1330 simulation ...
finished 518/1330 simulation ...
finished 519/1330 simulation ...
finished 520/1330 simulation ...
finished 521/1330 simulation ...
finished 522/1330 simulation ...
finished 523/1330 simulation ...
finished 524/1330 simulation ...
finished 525/1330 simulation ...
finished 526/1330 simulation ...
finished 527/1330 simulation ...
finished 528/1330 simulation ...
finished 529/1330 simulation ...
finished 530/1330 simulation ...
finished 531/1330 simulation ...
finished 5

finished 751/1330 simulation ...
finished 752/1330 simulation ...
finished 753/1330 simulation ...
finished 754/1330 simulation ...
finished 755/1330 simulation ...
finished 756/1330 simulation ...
finished 757/1330 simulation ...
finished 758/1330 simulation ...
finished 759/1330 simulation ...
finished 760/1330 simulation ...
finished 761/1330 simulation ...
finished 762/1330 simulation ...
finished 763/1330 simulation ...
finished 764/1330 simulation ...
finished 765/1330 simulation ...
finished 766/1330 simulation ...
finished 767/1330 simulation ...
finished 768/1330 simulation ...
finished 769/1330 simulation ...
finished 770/1330 simulation ...
finished 771/1330 simulation ...
finished 772/1330 simulation ...
finished 773/1330 simulation ...
finished 774/1330 simulation ...
finished 775/1330 simulation ...
finished 776/1330 simulation ...
finished 777/1330 simulation ...
finished 778/1330 simulation ...
finished 779/1330 simulation ...
finished 780/1330 simulation ...
finished 7

finished 1000/1330 simulation ...
finished 1001/1330 simulation ...
finished 1002/1330 simulation ...
finished 1003/1330 simulation ...
finished 1004/1330 simulation ...
finished 1005/1330 simulation ...
finished 1006/1330 simulation ...
finished 1007/1330 simulation ...
finished 1008/1330 simulation ...
finished 1009/1330 simulation ...
finished 1010/1330 simulation ...
finished 1011/1330 simulation ...
finished 1012/1330 simulation ...
finished 1013/1330 simulation ...
finished 1014/1330 simulation ...
finished 1015/1330 simulation ...
finished 1016/1330 simulation ...
finished 1017/1330 simulation ...
finished 1018/1330 simulation ...
finished 1019/1330 simulation ...
finished 1020/1330 simulation ...
finished 1021/1330 simulation ...
finished 1022/1330 simulation ...
finished 1023/1330 simulation ...
finished 1024/1330 simulation ...
finished 1025/1330 simulation ...
finished 1026/1330 simulation ...
finished 1027/1330 simulation ...
finished 1028/1330 simulation ...
finished 1029/

finished 1241/1330 simulation ...
finished 1242/1330 simulation ...
finished 1243/1330 simulation ...
finished 1244/1330 simulation ...
finished 1245/1330 simulation ...
finished 1246/1330 simulation ...
finished 1247/1330 simulation ...
finished 1248/1330 simulation ...
finished 1249/1330 simulation ...
finished 1250/1330 simulation ...
finished 1251/1330 simulation ...
finished 1252/1330 simulation ...
finished 1253/1330 simulation ...
finished 1254/1330 simulation ...
finished 1255/1330 simulation ...
finished 1256/1330 simulation ...
finished 1257/1330 simulation ...
finished 1258/1330 simulation ...
finished 1259/1330 simulation ...
finished 1260/1330 simulation ...
finished 1261/1330 simulation ...
finished 1262/1330 simulation ...
finished 1263/1330 simulation ...
finished 1264/1330 simulation ...
finished 1265/1330 simulation ...
finished 1266/1330 simulation ...
finished 1267/1330 simulation ...
finished 1268/1330 simulation ...
finished 1269/1330 simulation ...
finished 1270/

In [19]:
np.savetxt("List.csv", List,delimiter =", ", fmt ='% s')

In [20]:
np.savetxt("Kt_tension.csv", Kt_tension, delimiter =", ", fmt ='% s')
np.savetxt("Number_of_iter.csv", Number_of_iter, delimiter =", ", fmt ='% s')

In [31]:
List[559] # This one is highest iteration of 10 times

[0.018, 0.014]

In [33]:
def compute_stress_con(sat_holes_dia,sat_holes_loc):
    
    converge = 1
    iteration = 1
    stress_concentration = 1
    
    while converge >= 0.01: # less than 1 percent of the stress concentration convergence is needed
        mapdl.clear("NOSTART")
        mapdl.prep7()
        mapdl.units("SI")  # SI-International system (m, kg, s, K).

        # Define a PLANE183 element type with thickness
        mapdl.et(1, "PLANE183", kop3=3)
        mapdl.r(1, 0.001)  # Thickness of 1 mm

        # Define a material (nominal steel in SI)
        mapdl.mp("EX", 1, 210e9)  # Elastic moduli in Pa (kg/(m*s**2))
        mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
        mapdl.mp("NUXY", 1, 0.3)  # Poisson's Ratio
        mapdl.emodif("ALL", "MAT", 1)

        # Geometry
    
        # Create a rectangular area with the hole in the middle
        center_hole_dia = 0.045 # center hole diameter = 45 mm
    
        # Create the rectangle
        rect_anum = mapdl.blc4(width = 0.2, height = 0.1) # 200 mm x 100 mm

        # Create a circle in the middle of the rectangle
        circ_anum = mapdl.cyl4(0.1, 0.05, center_hole_dia/2)
    
        # Create a satellite circle at certain locations along y = 50 mm axis
        sat_circ_anum = mapdl.cyl4(sat_holes_loc ,0.05, sat_holes_dia/2)
        sat_circ_anum2 = mapdl.cyl4(0.2-sat_holes_loc ,0.05, sat_holes_dia/2) # 200 mm - left hole location = right hole location

        # Cut the circle out of the rectangle.
        area1= mapdl.asba(rect_anum, circ_anum)
        area2 = mapdl.asba(area1, sat_circ_anum)
        plate_with_hole_anum = mapdl.asba(area2, sat_circ_anum2)   
    
        # Meshing

        # Mesh the plate using a higher density near the hole and a lower density for the remainder of the plate
    
        mapdl.aclear("all")
  
        # Ensure there are at least 100 elements around the hole at beginning then increase 50 elements per iteration
        hole_esize = np.pi * center_hole_dia / ((iteration*50)+50) 
        sat_hole_esize = np.pi * sat_holes_dia / ((iteration*50)+50)  
        plate_esize = 0.01
    
        # Increased the density of the mesh at the center
        mapdl.lsel("S", "LINE", vmin=5, vmax=8) # center hole
        mapdl.lesize("ALL", hole_esize, kforc=1)
        mapdl.lsel("S", "LINE", vmin=9, vmax=40) # satellite holes
        mapdl.lesize("ALL", sat_hole_esize, kforc=1)
        mapdl.lsel("ALL")

        mapdl.esize(plate_esize)
        mapdl.amesh(plate_with_hole_anum)

        # Boundary Conditions

        # Fix the left-hand side of the plate in the X direction
        mapdl.nsel("S", "LOC", "X", 0)
        mapdl.d("ALL", "UX")

        # Fix a single node on the left-hand side of the plate in the Y direction
        mapdl.nsel("R", "LOC", "Y", 0.1 / 2)
        assert mapdl.mesh.n_node == 1
        mapdl.d("ALL", "UY")

        # Apply a force on the right-hand side of the plate.
        
        # Select the right-hand side of the plate.
        mapdl.nsel("S", "LOC", "X", 0.2)

        # Couple the DOF for these nodes
        mapdl.cp(5, "UX", "ALL")

        # Select a single node in this set and apply a force to it
        mapdl.nsel("r", "loc", "y", 0.1 / 2)
        mapdl.f("ALL", "FX", 1000)

        # Be sure to select all nodes again to solve the entire solution
        mapdl.allsel()

        # Solve the Static Problem

        mapdl.run("/SOLU")
        mapdl.antype("STATIC")
        mapdl.solve()

        # Post-Processing

        # Grab the stress from the result
        result = mapdl.result
        nnum, stress = result.principal_nodal_stress(0)
        von_mises = stress[:, -1]
        max_stress = np.nanmax(von_mises)

        # Compare to the "far field" stress by getting the mean value of the stress at the wall
        mask = result.mesh.nodes[:, 0] == 0.2
        far_field_stress = np.nanmean(von_mises[mask])

        # Adjust by the cross sectional area at the hole
        adj = 0.1 / (0.1 - 0.045) # 100 mm plate length, 45 mm center hole diameter
        stress_adj = far_field_stress * adj
        
        converge = abs(stress_concentration - (max_stress / stress_adj))/stress_concentration
        
        stress_concentration = max_stress / stress_adj
        
        iteration +=1
        
        print(iteration-1)
        print(converge)
        print(stress_concentration)
    return stress_concentration,iteration-1, converge

In [35]:
compute_stress_con(0.018,0.014)

1
1.0690406779702122
2.069040677970212
2
0.07535579831353736
2.22495489000184
3
0.012733688037715908
2.196623008534566
4
0.05319849131274194
2.079765978497671
5
0.06606571314635636
2.2171672010446493
6
0.04940855126920804
2.107620181719428
7
0.05152980082488224
2.2162254298979325
8
0.07261681744827708
2.055290192430805
9
0.013453088907306019
2.0829401941198906
10
0.0011463803003737586
2.0853280357252864


(2.0853280357252864, 10, 0.0011463803003737586)